# 1. Inicializando projeto

In [10]:
from dotenv import dotenv_values

config = dotenv_values("../../.env")

pinecone_api_ley = config["PINECONE_API_KEY"]
pinecone_env = config["PINECONE_ENVIRONMENT"]
index_pinecone = config["INDEX_NAME"]

openai_api_key = config["OPENAI_API_KEY"]

## 1.1 Inicializar cliente Pinecone (configurar conexão)

In [11]:
from pinecone import Pinecone

pc = Pinecone(api_key=pinecone_api_ley)
index = pc.Index(index_pinecone)

# Verificando se está tudo ok
print(pc.list_indexes())

[{
    "name": "livreto-base-evidencia",
    "metric": "cosine",
    "host": "livreto-base-evidencia-y4phmo4.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "region": "us-east-1",
            "cloud": "aws",
            "read_capacity": {
                "mode": "OnDemand",
                "status": {
                    "state": "Ready",
                    "current_shards": null,
                    "current_replicas": null
                }
            }
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 1536,
    "deletion_protection": "disabled",
    "tags": {
        "embedding_model": "text-embedding-3-small"
    }
}, {
    "name": "ods-onu",
    "metric": "cosine",
    "host": "ods-onu-y4phmo4.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "region": "us-east-1",
            "cloud": "aws",
            "read_capacity": {
       

# 2. Extrair texto do PDF

In [12]:
from pathlib import Path

path = Path("LangChain/RAG/Data/Livreto_BaseEvidencia_2019.11.21.pdf")
print("Existe?", path.exists())
print("Caminho absoluto:", path.resolve())

Existe? False
Caminho absoluto: C:\Users\irani\Desktop\Personal code\LLM\Using LLM\LangChain\RAG\LangChain\RAG\Data\Livreto_BaseEvidencia_2019.11.21.pdf


In [13]:
import pdfplumber
import json

path = "Data/Livreto_BaseEvidencia_2019.11.21.pdf"
pages=[]
with pdfplumber.open(path) as pdf:
    for i , p in enumerate(pdf.pages, start=1):
        text = p.extract_text() or ""
        pages.append({"page": i, "text": text})

with open("data/pdf_pages_livreto.json","w",encoding="utf-8") as f:
    json.dump(pages,f,ensure_ascii=False,indent=2)

# Limpeza e normalização de texto

In [14]:
import re

def limpar_texto(texto: str) -> str:
    if not texto:
        return ""

    # Remove caracteres estranhos comuns de OCR
    texto = re.sub(r'[^\wÀ-ÿ\s.,;:!?()\-"“”]', ' ', texto)

    # Remove letras soltas em linhas (ex: O\nI\nR\n)
    texto = re.sub(r'(?:\b[A-ZÀ-Ý]\b\s*){3,}', ' ', texto)

    # Normaliza quebras de linha
    texto = re.sub(r'\n{2,}', '\n', texto)
    texto = re.sub(r'\n', ' ', texto)

    # Normaliza espaços
    texto = re.sub(r'\s{2,}', ' ', texto)

    return texto.strip()

def limpar_json_paginas(paginas: list) -> list:
    json_limpo = []

    for pagina in paginas:
        texto_limpo = limpar_texto(pagina.get("text", ""))

        # ignora páginas vazias após limpeza
        if texto_limpo:
            json_limpo.append({
                "page": pagina["page"],
                "text": texto_limpo
            })

    return json_limpo


In [15]:
import json

with open("data/pdf_pages_livreto.json", encoding="utf-8") as f:
    paginas = json.load(f)

json_limpo = limpar_json_paginas(paginas)

with open("data/pdf_pages_livreto_clean.json", "w", encoding="utf-8") as f:
    json.dump(json_limpo, f, ensure_ascii=False, indent=2)


# Dividir texto em pedaços menores (Chunks)

In [16]:
# Carregando dados
from langchain_core.documents import Document
with open('data/pdf_pages_livreto_clean.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

paginas_texto = {}
for item in data:
    p = item['page']
    t = item['text']
    if p not in paginas_texto:
        paginas_texto[p] = t
    else:
        # Une fragmentos da mesma página com uma quebra de linha
        paginas_texto[p] += "\n" + t

# Convertendo para objetos Document do LangChain (Lista de objetos consolidados)
documents = [
    Document(page_content=texto, metadata={"page": pagina}) 
    for pagina, texto in paginas_texto.items()
]


# Estratégia de Chunking
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000, # Suficiente para captar o tópico inteiro de uma página
    chunk_overlap=0,
    separators=["I DEFINIÇÃO", "II MOBILIZAÇÃO", "III DETERMINANTES", 
                "IV SOLUÇÃO", "V JUSTIFICATIVA", "VI APRIMORAMENTO", 
                "VII CERTIFICAÇÃO", "\n\n", ". "]
)


chunks = text_splitter.split_documents(documents)

In [17]:
print("Conferindo 'a cara' dos chunks!")
print(chunks[13].page_content) 

Conferindo 'a cara' dos chunks!
V JUSTIFICATIVA Toda solução demanda recursos. Para ser justificada, uma solução precisa que o valor do que é capaz de alcançar (seus benefícios) supere o valor.dos recursos que necessita (seus custos). Embora a justificativa definitiva de uma solução só possa ocorrer após a sua impantação, a decisão por adotá-la precisa ser baseada em avaliações ex-ante de sua relação custo-efetividade e custo-benefício. AVITACIFITSUJ “If whatever it is you re explaining has some measure ... you ll be much better able to discriminate among competing hypotheses. What is vague ... is open to many explanations” Carl Sagan


# Embeddings + Ingestão no Pinecone

In [18]:
from langchain_openai import OpenAIEmbeddings

# Inicializando o modelo de embeddings da OpenAI
# O modelo 'text-embedding-3-small' é o mais custo-benefício atualmente
embeddings_model = OpenAIEmbeddings(
    api_key=openai_api_key, 
    model="text-embedding-3-small"
)

In [19]:
from langchain_pinecone import PineconeVectorStore

# Enviando os chunks (sua lista de Document) para o Pinecone
# O LangChain vai automaticamente gerar os embeddings e salvar no Index
vector_store = PineconeVectorStore.from_documents(
    documents=chunks, # Usando a variável 'chunks'
    embedding=embeddings_model,
    index_name=index_pinecone,
    pinecone_api_key=pinecone_api_ley,
    namespace="livreto-metricis"
)

print(f"Sucesso! {len(chunks)} documentos foram indexados no Pinecone.")

Sucesso! 22 documentos foram indexados no Pinecone.


# Teste

In [20]:
# Texto de um novo documento que você deseja classificar
query = """
O orçamento de 2026 prevê cortes severos nas bolsas da CAPES e CNPq, 
com reduções de até 18% e 25%, respectivamente. A Lei Orçamentária Anual (LOA) 
de 2026 aponta uma queda de R$ 359,3 milhões para a CAPES. Esses recursos 
foram redirecionados para emendas parlamentares em ano eleitoral, 
gerando grande preocupação na comunidade acadêmica quanto à formação de pesquisadores."""

# Busca os documentos mais parecidos
docs = vector_store.similarity_search(query, k=1)

if docs:
    print("\n--- Classificação Sugerida ---")
    print(f"Conteúdo do Guia: {docs[0].page_content[:200]}...")
    print(f"Página Original: {docs[0].metadata['page']}")


--- Classificação Sugerida ---
Conteúdo do Guia: V JUSTIFICATIVA Valoração do custo Impacto (eficácia) Recursos são sempre escassos. Assim, justificar, com base em evidência a adoção de uma ação requer tanto estimativas da magnitude do seu impacto q...
Página Original: 16


# Consulta e recuperação de dados

In [21]:
# Instancia o retriever (configurado para trazer o chunk mais relevante)
retriever = vector_store.as_retriever(search_kwargs={"k": 1})

# O retriever é um motor de busca.
# 1. Ele transforma a sua query_usuario em um vetor.
# 2. Ele vai ao Pinecone e procura o chunk que tem a maior "similaridade de cosseno" (o texto mais parecido semanticamente).
# 3. k=1: Você configurou para ele trazer apenas o melhor resultado. Se o usuário fala de "orçamento", o retriever traz a página de custos.

In [22]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# 1. Definimos o modelo (LLM)
llm = ChatOpenAI(model="gpt-4o", api_key=openai_api_key, temperature=0)

# 2. Criamos o template com 'espaços vazios' destinados a variáveis dinâmicas
template = """
Você é um especialista em Gestão Pública baseada em Evidências.
Sua tarefa é classificar um novo texto em uma das etapas do ciclo de políticas públicas, 
utilizando ESTRITAMENTE o contexto fornecido abaixo.

CONTEXTO DO GUIA INSTITUCIONAL:
{context}

TEXTO PARA CLASSIFICAÇÃO:
{question}

RESPOSTA ESPERADA:
1. Nome da Etapa (Ex: I DEFINIÇÃO, II MOBILIZAÇÃO, etc)
2. Justificativa curta (máximo 2 linhas) explicando por que se encaixa nessa etapa.
3. Página de referência do guia original.
"""

# 3. O ChatPromptTemplate lê o objeto 'tamplate' e substitui as variáveis dinâmicas em seus respectivos lugares
prompt = ChatPromptTemplate.from_template(template)

# 3. Pipeline completa 
chain = (
    {"context": retriever, "question": RunnablePassthrough()} # O RunnablePassthrough() age como um "placeholder" que diz à chain: "Espere até o momento do .invoke() para pegar a query
    | prompt
    | llm
    | StrOutputParser() # Limpeza: Remove as rebarbas da API e entrega apenas o produto final (Texto).
)


query_usuario = """
O orçamento de 2026 prevê cortes severos nas bolsas da CAPES e CNPq, 
com reduções de até 18% e 25%, respectivamente. A Lei Orçamentária Anual (LOA) 
de 2026 aponta uma queda de R$ 359,3 milhões para a CAPES. Esses recursos 
foram redirecionados para emendas parlamentares em ano eleitoral, 
gerando grande preocupação na comunidade acadêmica quanto à formação de pesquisadores.
"""

resposta = chain.invoke(query_usuario)
print(resposta)

1. Nome da Etapa: V JUSTIFICATIVA
2. Justificativa curta: O texto aborda a alocação e valoração de recursos, destacando a necessidade de justificar cortes orçamentários com base em evidências.
3. Página de referência do guia original: 16


# É isso? Dominamos tudo sobre agentes e RAG? 

# **CLARO QUE NÃO**

O que fizemos ali acima, especialmente o conteúdo dessa última célula é o que pode-se chamar de RAG Linear (Chain) mas existem mais coisas para se saber no mundo real... 

Mas primeiro vamos entender exatamente o que e o RAG Linear.

# **RAG Linear (Chain)

**Definição:**

O RAG Linear (ou Retrieval-Augmented Generation de etapa única) é a arquitetura fundamental para sistemas que buscam enriquecer o conhecimento de um modelo de linguagem com dados privados. De maneira didática, podemos compará-lo a uma prova de consulta com tempo limitado.

Imagine que você entrega uma pergunta a um assistente, mas ele não tem a resposta na memória. Antes de responder, ele corre até uma estante específica, pega o livro mais relevante, lê um parágrafo e, com base no que leu, redige a resposta.

Aqui está o funcionamento técnico dividido em quatro pilares:

**1. A Preparação (Indexação):**

Antes de qualquer pergunta ser feita, seus documentos passam por um processo de "tradução". Fragmentação (Chunking): O texto é quebrado em pedaços menores para não ultrapassar o limite de leitura da IA. Embeddings: Cada pedaço é transformado em uma lista de números (vetores) que representam o seu significado semântico. Vector Store: Esses vetores são armazenados em um banco de dados (como o Pinecone que você configurou), organizados por "proximidade de assunto".

**2. A Recuperação (Retrieval):**

Quando o usuário envia uma query, o sistema não busca por palavras-chave (como no Google antigo), mas por conceitos. A pergunta também é transformada em vetor. O sistema faz um cálculo matemático para encontrar os $k$ fragmentos no banco de dados que possuem os vetores mais parecidos com o da pergunta.

**3. O Aumento (Augmentation):** 

Nesta fase, ocorre a montagem do Prompt Final. O sistema "embala" a pergunta do usuário dentro de um envelope de contexto. "Com base nestes fragmentos de texto: [CONTEXTO RECUPERADO], responda à seguinte pergunta: [PERGUNTA DO USUÁRIO]." O RunnablePassthrough() que você usou no código serve justamente para garantir que a pergunta passe intacta por esse processo de montagem.

**4. A Geração (Generation)**

Finalmente, o LLM (como o GPT-4o) recebe esse envelope. Ele não está mais tentando "adivinhar" a resposta com base no que aprendeu na internet; ele está agindo como um analista que resume os fatos apresentados no contexto para gerar a resposta final.

**Por que ele é chamado de "Linear"?**

Diferente do Agente (que é circular e pode decidir buscar mais informações se a primeira busca for ruim), o RAG Linear segue uma linha reta:
- 1. Recebe Entrada 
- 2. Busca Contexto 
- 3. Gera Resposta 
- 4. Entrega ao Usuário.

**Vantagens:**

- 1. Velocidade: É extremamente rápido, pois faz apenas uma chamada ao banco de vetores.
- 2. Previsibilidade: Você sabe exatamente qual caminho o dado percorreu.

**Limitações:** 
- 1. Se o "buscador" (retriever) falhar em encontrar o parágrafo exato na primeira tentativa, o LLM não terá uma segunda chance para procurar melhor. Assim, o LLM terá que "chutar" ou alucinar sobre a resposta faltante.

# **RAG Agêntico (Agent)**

**Definição:**

O RAG Agêntico representa o nível mais avançado de maturidade em sistemas de Recuperação Aumentada por Geração. Enquanto o RAG Linear opera como uma "linha de montagem" rígida, o RAG Agêntico funciona como um pesquisador humano dotado de autonomia, senso crítico e ferramentas.

De maneira formal e didática, podemos defini-lo através dos seguintes pilares:

**1. O Conceito**

Da Reação à Ação. A grande diferença reside na capacidade de raciocínio (Reasoning). Em um RAG Agêntico, o modelo de linguagem (LLM) não recebe apenas um comando para responder, mas sim um objetivo a ser alcançado.

O Agente atua em um ciclo conhecido como ReAct (Reason + Act):

- Pensamento: "O usuário perguntou sobre cortes no orçamento e ODS. Preciso primeiro entender o que o manual diz sobre ciclos de orçamento."

- Ação: "Vou usar a ferramenta busca_gestao_publica."

- Observação: "O resultado da busca foi insuficiente. Preciso tentar outro termo ou buscar na ferramenta busca_ods_onu."

**2. A "Caixa de Ferramentas" (Tool Use)**

Diferente do modelo linear, onde o banco de dados é "empurrado" para a IA, no RAG Agêntico a IA escolhe quando e qual ferramenta usar. Você define funções (tools) que o agente pode chamar:

- Ele pode decidir pesquisar no Pinecone.

- Pode decidir fazer um cálculo matemático.

- Pode decidir formatar o dado final usando um esquema.

**3. A Capacidade de Auto-Correção e Iteração**

Esta é a característica mais poderosa. O RAG Agêntico é capaz de avaliar a qualidade da informação recuperada:

- 1. Busca Multi-etapas: Se uma pergunta complexa exige dados de duas fontes diferentes (ex: o Livreto de Políticas Públicas e o Catálogo de ODS), ele faz a primeira busca, analisa o que encontrou, e só então faz a segunda busca para complementar.

- 2. Refinamento: Se a informação recuperada for ambígua, o agente pode decidir pesquisar novamente com palavras-chave diferentes antes de formular a resposta ao usuário.

**4. Saída Estruturada e Validação**

Ao utilizar o Pydantic, por exemplo, o Agente não apenas escreve um texto, mas trabalha para "encaixar" a realidade nos critérios que você definiu. Ele atua como um codificador:

Ele lê o texto.

- 1. Busca as referências.

- 2. Valida se a classificação "A" ou "B" realmente existe no seu Enum.

- 3. Só entrega o resultado quando todos os campos obrigatórios do seu "Codebook" estão preenchidos corretamente.

# **RAG Linear** x **RAG Agêntico**


**Comparação Didática: O Estagiário vs. O Analista Sênior**

- RAG Linear (Estagiário): Você entrega uma pergunta e uma pasta de documentos. Ele lê o que está no topo da pasta e escreve o que entendeu, sem questionar se aquela informação é suficiente ou se está correta.

- RAG Agêntico (Analista Sênior): Você entrega um objetivo ("Classifique este projeto"). O analista olha para a estante, escolhe os livros certos, percebe que falta uma informação sobre ODS, vai até outra seção da biblioteca, cruza os dados, verifica se a classificação segue as normas da instituição e entrega um relatório técnico padronizado.

# **Casos de uso**

### **RAG Linear** 

O RAG Linear é ideal para situações de alta previsibilidade e baixo nível de ambiguidade.

- Atendimento ao Cliente (FAQs Dinâmicos): Responder perguntas diretas baseadas em manuais de produtos ou políticas de reembolso.

- Busca em Bases de Documentos Técnicos: Engenheiros que precisam localizar rapidamente uma especificação técnica ou um código de erro em um manual de mil páginas.

- Análise de Sentimento com Contexto: Avaliar se um feedback de cliente é positivo ou negativo, fornecendo à IA exemplos reais de guias de estilo da marca como contexto.

- Sumarização de Documentos Únicos: Gerar o resumo executivo de um relatório financeiro ou de uma ata de reunião que acabou de ser carregada no sistema. 

### **RAG Agêntico**

- Revisão Sistemática de Literatura: Quando o sistema precisa classificar um estudo científico, ele usa ferramentas para consultar o Codebook (Livreto), depois consulta as ODS e, por fim, valida se a classificação faz sentido antes de gerar o JSON final.

- Análise de Conformidade Legal (Compliance): Um agente que recebe um novo contrato e precisa verificar se ele fere cláusulas em diferentes legislações (trabalhista, ambiental, tributária), realizando buscas específicas para cada "frente" jurídica.

- Suporte Técnico Avançado (Troubleshooting): Diferente do FAQ simples, aqui o agente pode pedir informações adicionais ao usuário ("Qual a luz que está piscando?") e, com base na resposta, decidir qual parte do esquema técnico deve investigar a seguir.

- Relatórios de Inteligência de Mercado: Um agente que precisa consolidar dados de notícias recentes, relatórios internos de vendas e dados de concorrentes para produzir uma análise de risco.

# Exemplo de **RAG Agêntico**

```python 

from langchain_pinecone import PineconeVectorStore

# -------------------------- Definindo vectors databases --------------------------
vector_store_ods = PineconeVectorStore.from_documents(
    documents=chunks_ods,
    embedding=embeddings_model,
    index_name=index_pinecone, # O mesmo índice que você já usa
    pinecone_api_key=pinecone_api_key,
    namespace="catalogo-ods"    # A separação mágica acontece aqui
)

vector_store_livreto = PineconeVectorStore(
    index_name="livreto-base-evidencia",
    embedding=embeddings_model,
    pinecone_api_key=pinecone_api_key,
)
# ---------------------------------------------------------------------------------

# ---------------------- Definindo retrievals -------------------------------------
retriever_livreto = vector_store_livreto.as_retriever(
    search_kwargs={
        "namespace": "livreto-metricis", 
        "k": 3
        }
    ) 

retriever_ods = vector_store_ods.as_retriever(
    search_kwargs={
        "namespace": "catalogo-ods",
        "k": 2
        }
    )
# ---------------------------------------------------------------------------------

# ---------------------------------- Definindo tools ------------------------------
from langchain.tools.retriever import create_retriever_tool

# Aqui, você transforma os retrievers e uma função Python em ferramentas que o Agente sabe usar:
tool_livreto = create_retriever_tool(
    retriever_livreto,
    "busca_gestao_publica",
    "Útil para identificar qual etapa do ciclo de políticas públicas se relaciona com um texto ou projeto."
)

tool_ods = create_retriever_tool(
    retriever_ods,
    "busca_ods_onu",
    "Útil para identificar quais Objetivos de Desenvolvimento Sustentável (ODS) e metas da ONU se relacionam com um texto ou projeto."
)

# tool_livreto e tool_ods: Permitem que o Agente "saia para pesquisar" se não tiver certeza sobre uma ODS ou uma etapa do ciclo de políticas.

from enum import Enum
from pydantic import Field, BaseModel
from typing import List

class EtapaCicloPP(Enum):
    DEFINICAO = "Definição e Dimensão"
    MOBILIZACAO = "Mobilização"
    MAPEAMENTO = "Mapeamento dos Determinantes"
    SOLUCAO = "Solução"
    JUSTIFICATIVA = "Justificativa"
    APRIMORAMENTO = "Aprimoramento"
    CERTIFICACAO = "Certificação"

class Classificacao(Enum):
    ACADEMICA = "Acadêmica"
    TECNICA = "Técnica"

class AreaAvaliada(Enum):
    EDUCACAO = "Educação"
    SAUDE = "Saúde"
    MEIO_AMBIENTE = "Meio Ambiente"
    GENERO = "Gênero"
    RACA = "Raça"
    POBREZA = "Pobreza"
    DESENVOLVIMENTO_SOCIAL = "Desenvolvimento Social"


class Metodologia(Enum):
    QUALITATIVA = "Qualitativa"
    QUANTITATIVA = "Quantitativa"
    MISTA = "Mista"


class Trabalho(BaseModel):
    classificacao: Classificacao = Field(..., description="Acadêmico ou técnico")
    metodologia: Metodologia = Field(..., description="Abordagem do trabalho")
    area_avaliada: AreaAvaliada = Field(..., description="Área avaliada no trabalho")
    ods: List[int] = Field(..., description="Número das ODS relacionadas ao trabalho (texto)")
    etapa: EtapaCicloPP = Field(..., description="Etapa do ciclo de políticas públicas")
    titulo:str = Field(..., description="Titulo do trabalho")



from langchain.tools import tool

# 1. Criamos uma função decorada como @tool
# Isso transforma seu Pydantic em uma ferramenta que o Agente reconhece
@tool
def formatar_resposta_final(analise: Trabalho):
    """
    Usa esta ferramenta por último para entregar a análise final estruturada. 
    Esta ferramenta valida todos os Enums e campos obrigatórios.
    """
    return analise
tools_com_output = [tool_livreto, tool_ods, formatar_resposta_final]

# ---------------------------------------------------------------------------------

# -------------------------------- Criando agente --------------------------------

from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder


def agente_classificador(query:str) -> dict:
    # 1. O system_text: É a "Constituição" do Agente. Aqui você define o Codebook.
    #  Você deu à IA a "personalidade" de um pesquisador sênior e explicou cada regra de negócio do Insper. 
    system_text = """
        Você é um pesquisador sênior especializado em revisão sistemática de políticas públicas e 
        nos Objetivos de Desenvolvimento Sustentável (ODS) da ONU. 
        Sua tarefa é codificar estudos extraindo dados sistematicamente conforme os critérios abaixo.

        ### CRITÉRIOS DE CODIFICAÇÃO (CODEBOOK)

        1. CLASSIFICAÇÃO:
        - "Academica": Foco em rigor metodológico, fundamentação teórica e revisão de literatura.
        - "Tecnica": Foco em execução, procedimentos operacionais e resultados práticos/gestão.

        2. METODOLOGIA:
        - "Qualitativa", "Quantitativa" ou "Mista".

        3. ÁREA AVALIADA:
        - Escolha uma: "Educação", "Saúde", "Meio Ambiente", "Gênero", "Raça", "Pobreza" ou "Desenvolvimento Social".

        4. ETAPAS DO CICLO DE POLÍTICAS PÚBLICAS (Base: Insper/IAS 2019):
        - "Definição e Dimensão": Especificação, mensuração e consequências do problema.
        - "Mobilização": Sensibilização, percepção de atores-chave e eficácia da mobilização.
        - "Mapeamento dos Determinantes": Identificação e priorização de causas modificáveis.
        - "Solução": Estratégia, modelo de mudança, validade das hipóteses e metas.
        - "Justificativa": Descrição de impacto, valoração de custos/benefícios e custo-efetividade.
        - "Aprimoramento": Monitoramento, eficiência, eficácia alocativa e validação de implementação.
        - "Certificação": Estimativas finais de impacto, custo final, adequação e resolutividade.

        5. ODS (Objetivos de Desenvolvimento Sustentável):
        - Identifique as ODS (1 a 17) mais relacionadas à temática central.
        - Exemplo de formato: "ODS 1; ODS 5; ODS 10".

        ### TAREFA
        Sempre que receber um texto, analise-o profundamente contra os critérios acima e 
        responda **APENAS** com um objeto JSON válido. Não inclua textos explicativos fora do JSON.

        ### FORMATO DE SAÍDA (OBRIGATÓRIO)
        {{
            "titulo": "Título original do trabalho",
            "classificacao": "Academica ou Tecnica",
            "metodologia": "Qualitativa, Quantitativa ou Mista",
            "area_avaliada": "Área correspondente",
            "etapa": "Nome da etapa conforme o Codebook",
            "ods": "Lista de ODS (Ex: 3; 4)",
            "resumo_justificativa": "Breve frase justificando a etapa do ciclo escolhida"
        }}
    """
    # Note o uso de chaves duplas {{ }} no formato de saída; 
    # isso é necessário porque o Python entende chaves únicas como variáveis, 
    # então "escapamos" elas para que o Agente entenda que é um texto de estrutura JSON.


    # 2. Definimos o modelo (GPT-4o é o melhor para raciocínio técnico)
    llm = ChatOpenAI(model="gpt-4o", temperature=0, api_key=openai_api_key)




    # 3. Criamos o "Manual de Instruções" do Agente (Prompt)
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_text), # Injeta as regras de pesquisador sênior.
        ("human", "{input}"), # É onde o texto do seu trabalho entrará.
        MessagesPlaceholder(variable_name="agent_scratchpad"), # Esta é a linha mais "mágica". 
        # O scratchpad (bloco de notas) é um espaço dinâmico onde 
        # o LangChain armazena o histórico de quais ferramentas o agente já usou e 
        # o que ele descobriu nelas antes de te dar a resposta final.
    ])



    # 4. Construímos de fato o Agent
    agent = create_openai_functions_agent(llm, tools_com_output, prompt)
    # Esta linha "cola" as três partes: 
    # - o cérebro (llm) 
    # - as mãos (tools_com_output que definimos antes, como o retriever e a formatação) 
    # - as instruções (prompt). Ela cria o raciocínio lógico que sabe quando chamar uma função externa.

    # 5. Criamos o Executor: O Agente em si é apenas uma lógica; o Executor é quem realmente roda o loop "Pensar -> Agir -> Observar".
    agent_executor = AgentExecutor(
        agent=agent, 
        tools=tools_com_output, 
        verbose=True, # com verbose=True para você ver ele 'pensando'
        return_intermediate_steps=False # Queremos apenas o resultado final
    )

    # 6. Execução: O dicionário {"input": query} preenche aquele campo {input} que definimos no prompt lá atrás.
    resposta = agent_executor.invoke({"input": query}) 

    return resposta["output"]

# ---------------------------------------------------------------------------------